#### Friday, April 19, 2024

mamba activate langchain3

Local model will be [lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF](https://huggingface.co/lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF)

Switched to [QuantFactory/Meta-Llama-3-8B-Instruct-GGUF](https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF)

This all runs locally. Ran each invoke method 3 times and saved the output to show the differences in the response.

In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="I'm a chatbot, oh so fine,\nCreated to assist, all the time!\nMy name is LLaMA, nice to meet you too,\nI'll respond in rhymes, that's what I love to do!", role='assistant', function_call=None, tool_calls=None)


Another LLama 3 8B Instruct quantized model. [QuantFactory/Meta-Llama-3-8B-Instruct-GGUF](https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF)

In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="QuantFactory/Meta-Llama-3-8B-Instruct-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Nice to meet you, don't be shy,\nI'm an AI assistant, flying high!\nI'll help with tasks and answers provide,\nIn rhyming form, I'll be your guide!", role='assistant', function_call=None, tool_calls=None)


# Retrieval

This notebook covers basic walkthrough of retrieval functionality in LangChain. For more information, see:

- [Retrieval Documentation](https://python.langchain.com/docs/modules/data_connection/)

- [Advanced Retrieval Types](https://python.langchain.com/docs/modules/data_connection/retrievers/)

- [QA with RAG Use Case Documentation](https://python.langchain.com/docs/use_cases/question_answering/)

## Load Documents

In [25]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [26]:
len(docs)

1

## Split documents

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [28]:
len(documents)

3

In [29]:
type(documents)

list

In [30]:
documents[0]

Document(page_content='Getting started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Introduction', 'language': 'en'})

## Index Documents

In [31]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

We want to use [HuggingFace embeddings](https://python.langchain.com/docs/integrations/text_embedding/huggingfacehub/).

In [32]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

In [33]:
# embeddings = OpenAIEmbeddings()
#embeddings = OpenAIEmbeddings(base_url="http://localhost:1234/v1", api_key="lm-studio")
embeddings = HuggingFaceEmbeddings()

In [34]:
vector = FAISS.from_documents(documents, embeddings)

## Query Documents


In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [36]:
# llm = ChatOpenAI()
llm = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [37]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [38]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()

In [39]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [41]:
# the call to the llm is made with the invoke method.
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# *** Run 1 ***
# Based on the provided context, LangSmith helps with testing by providing a platform for building production-grade LLM applications and allowing you to closely monitor and evaluate your application. It offers features such as tracing, evaluation, and prompt management.

# In particular, LangSmith provides a built-in accuracy evaluator that can be used to grade the results of an AI system. This is demonstrated in the code examples provided, where an `exact_match` evaluator is defined to score the output of the AI system against expected outputs.

# Additionally, LangSmith allows you to create datasets and examples, which can be used as test cases for evaluating your AI system's performance. The `runOnDataset` function can be used to run your AI system on a dataset and generate results, which can then be evaluated using custom evaluators like the one defined in the example.

# Overall, LangSmith provides a range of tools and features that can help with testing and evaluating LLM applications, making it easier to develop and deploy high-quality AI systems.


# *** Run 2 ***
# Based on the provided context, LangSmith seems to provide features for building production-grade LLM applications and closely monitoring and evaluating them. Specifically, it appears that LangSmith helps with testing by:

# 1. **Evaluating results**: LangSmith provides a built-in accuracy evaluator, as shown in the Python example, which allows you to test and grade the output of your AI system.
# 2. **Creating datasets**: You can create sample datasets in LangSmith, which are used for testing and evaluating your AI system's performance.
# 3. **Running experiments**: LangSmith allows you to run experiments with your AI system, using a dataset and evaluators to score the results.

# Overall, LangSmith seems to provide a platform for building and testing LLM applications, making it easier to develop and deploy AI-powered solutions.


# *** Run 3 ***
# According to the provided context, LangSmith can help with testing by providing a platform for building production-grade LLM applications and closely monitoring and evaluating them. This allows developers to ship their applications quickly and with confidence.

# LangSmith provides features such as tracing, evaluation, and prompt management, which can be used to test and grade the performance of an AI system. For example, LangSmith's built-in accuracy evaluator can be used to evaluate the results of an AI system against a set of test cases, providing feedback on its performance.

# Additionally, LangSmith allows developers to create custom evaluators using JavaScript or Python code, which can be used to implement complex testing scenarios and evaluation criteria.





According to the provided context, LangSmith can help with testing by providing a platform for building production-grade LLM applications and closely monitoring and evaluating them. This allows developers to ship their applications quickly and with confidence.

LangSmith provides features such as tracing, evaluation, and prompt management, which can be used to test and grade the performance of an AI system. For example, LangSmith's built-in accuracy evaluator can be used to evaluate the results of an AI system against a set of test cases, providing feedback on its performance.

Additionally, LangSmith allows developers to create custom evaluators using JavaScript or Python code, which can be used to implement complex testing scenarios and evaluation criteria.


## Advanced Retrieval

In [20]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=llm)

In [21]:
retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [24]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# *** Run 1 ***
# Based on the provided context, LangSmith can help with testing by allowing you to:

# 1. **Create a dataset**: You can create a sample dataset in LangSmith and define test cases for your AI system.
# 2. **Evaluate results**: LangSmith provides built-in evaluators, such as accuracy evaluator, or allows you to define custom evaluators to grade the results of your AI system.
# 3. **Run experiments**: You can run experiments on your AI system using LangSmith's `runOnDataset` method, which takes in a dataset and an AI system function as inputs.
# 4. **Monitor and evaluate performance**: LangSmith provides tracing capabilities to monitor the performance of your AI system and evaluate its results.

# In the provided example, you can see how to create a dataset, define a custom evaluator (`exact_match`), and run an experiment using `runOnDataset`. The result of the experiment is then evaluated using the defined evaluator.



# *** Run 2 ***
# According to the provided context, LangSmith helps with testing by providing a platform for building production-grade LLM applications and closely monitoring and evaluating them. It allows you to create datasets, define test cases (examples), and optionally use evaluators to grade the results.

# For example, in the code snippet provided, it demonstrates how to:

# 1. Create a dataset and add examples to it.
# 2. Define an evaluator function (`exact_match`) that checks if the output of the AI system matches the expected output.
# 3. Use the `evaluate` function to run the AI system on the dataset and score its performance using the defined evaluator.

# This allows you to test and evaluate your LLM application's performance and make improvements before deploying it in production.



# *** Run 3 ***
# Based on the provided context, LangSmith can help with testing by providing a platform to closely monitor and evaluate LLM applications. It allows you to define datasets, create examples, and use evaluators to grade the results. This enables you to test your AI system and get feedback on its performance.

# In the example code, it is shown how to create a dataset, create examples, define an evaluator (in this case, `exact_match`), and then run the experiment using the `runOnDataset` function. The `evaluate` function is used to score the results based on the defined evaluators.

# LangSmith also provides features such as tracing, which allows you to log traces of your application's execution, and prompt hub, a tool for managing prompts in your LLM applications. These features can help with testing by providing insights into how your AI system is performing and identifying areas that need improvement.






Based on the provided context, LangSmith can help with testing by providing a platform to closely monitor and evaluate LLM applications. It allows you to define datasets, create examples, and use evaluators to grade the results. This enables you to test your AI system and get feedback on its performance.

In the example code, it is shown how to create a dataset, create examples, define an evaluator (in this case, `exact_match`), and then run the experiment using the `runOnDataset` function. The `evaluate` function is used to score the results based on the defined evaluators.

LangSmith also provides features such as tracing, which allows you to log traces of your application's execution, and prompt hub, a tool for managing prompts in your LLM applications. These features can help with testing by providing insights into how your AI system is performing and identifying areas that need improvement.
